In [2]:
from modAL.models import ActiveLearner
from modAL.acquisition import max_EI
from modAL.disagreement import max_std_sampling, vote_entropy_sampling
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C ,WhiteKernel as Wht,Matern as matk
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    mean_absolute_percentage_error,
    mean_absolute_error,
)

In [6]:
def gpregression(Xtrain,Ytrain,Nfeature):    
    cmean=[1.0]*Nfeature
    cbound=[[1e-6, 1e6]]*Nfeature
    kernel = C(1.0, (1e-6,1e6)) * matk(cmean,cbound,1.2) + Wht(1.0, (1e-6, 1e6))  # Matern kernel
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=False)
    gp.fit(Xtrain, Ytrain)
    return gp

def gprediction(gpnetwork,xtest):
    y_pred, sigma = gpnetwork.predict(xtest, return_std=True)
    return y_pred, sigma


In [5]:
def upperConfidenceBound(xdata,gpnetwork,epsilon):
    """
        xdata: input feature vectors or PCs of the REMAINING set
        gpnetwork: GPR model
        epsilon: control exploration/exploitation. Higher epsilon means more exploration
    """
    ye_pred, esigma = gprediction(gpnetwork, xdata)
    ucb = np.empty(ye_pred.size, dtype=float)
    for ii in range(0,ye_pred.size):
        if esigma[ii] > 0:
            ucb[ii]=(ye_pred[ii]+epsilon*esigma[ii])
        else:
            ucb[ii]=0.0
    return ucb

In [4]:
train_data = pd.read_csv("../data/processed/train.csv")
val_data = pd.read_csv("../data/processed/val.csv")
train_X, train_y = train_data.drop("pt_loss", axis=1), train_data["pt_loss"]
val_X, val_y = val_data.drop("pt_loss", axis=1), val_data["pt_loss"]
n_start_points = 10
n_steps = 10

In [ ]:
ndata = train_X.values.shape[0]   # Length of all data or number of molecules in the dataset
n_PC = 7
Xdata = train_X.values

ntrainInit = 10 # Number of initial training data points
nremain = ndata - ntrainInit
np.random.seed(10) # Fix the seed value
dataset = np.random.permutation(ndata)
a1data = np.empty(ntrainInit, dtype=int)
a2data = np.empty(nremain, dtype=int)
a1data[:] = dataset[0:ntrainInit]
a2data[:] = dataset[ntrainInit:ndata]

Xtrain = np.ndarray(shape=(ntrainInit, n_PC), dtype=float)
XtrainSmiles = np.chararray(ntrainInit, itemsize=100)
Ytrain = np.zeros(ntrainInit, dtype=float)
Xtrain[0:ntrainInit, :] = Xdata[a1data, :]
XtrainSmiles[0:ntrainInit] = Xsmiles[a1data]

Xremain = np.ndarray(shape=(nremain, n_PC), dtype=float)
XremainSmiles = np.chararray(nremain, itemsize=100)
Yremain = np.empty(nremain, dtype=float)
Xremain[0:nremain, :] = Xdata[a2data, :]
XremainSmiles[0:nremain] = Xsmiles[a2data]

print('*** Initial training set ***')
print(115*'-')
print('{:<5s}{:<80s}{:<15s}'.format('ID','SMILES','Eox (V vs. NHE)'))
print(115*'-')
for i in range(ntrainInit):
    print('{:<5d}{:<80s}{:<15f}'.format(i,XtrainSmiles[i].decode(),Ytrain[i]))
print(115*'-')  
print("Total number of inital training points: ", ntrainInit)
print("Total number of remaining points: ", nremain)

In [12]:
train_idx = np.random.choice(len(train_X), n_start_points)

In [17]:
train_xb = train_X.iloc[train_idx].values
train_yb = train_y.iloc[train_idx].values

In [18]:
train_xp = train_X.drop(train_idx, axis=0).values
train_yp = train_y.drop(train_idx, axis=0).values

In [26]:
learner = ActiveLearner(
    estimator=SGDRegressor(),
    query_strategy=max_EI,
    X_training=train_xb,
    y_training=train_yb
)

In [27]:
for idx in range(n_steps):
    query_idx, query_instance = learner.query(train_xp)
    learner.teach(
        X=train_xp[query_idx],
        y=train_yp[query_idx].reshape(1, )
    )
    # remove queried instance from pool
    train_xp = np.delete(train_xp, query_idx, axis=0)
    train_yp = np.delete(train_yp, query_idx)
    preds = learner.predict(val_X.values)
    print('R2 after query no. %d: %f' % (idx+1, r2_score(val_y, preds)))

TypeError: BaseSGDRegressor.predict() got an unexpected keyword argument 'return_std'